In [1]:
using Slepians, FFTW, Plots, LinearAlgebra, Arpack

┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.9.2). Unexpected behavior may occur.
└ @ ~/Documents/Repos/Slepians.jl/Examples/Manifest.toml:0
[ Info: Precompiling Slepians [97caf2bd-9cc0-4738-8821-c8999f46f3c9]
Precompiling Plots
  ✓ IniFile
  ✓ LaTeXStrings
  ✓ StatsAPI
  ✓ NaNMath
  ✓ Formatting
  ✓ OrderedCollections
  ✓ InverseFunctions
  ✓ URIs
  ✓ DataAPI
  ✓ Scratch
  ✓ Graphite2_jll
  ✓ OpenSSL_jll
  ✓ Libmount_jll
  ✓ LLVMOpenMP_jll
  ✓ Xorg_libXau_jll
  ✓ Bzip2_jll
  ✓ libpng_jll
  ✓ libfdk_aac_jll
  ✓ LAME_jll
  ✓ RecipesBase
  ✓ LERC_jll
  ✓ MacroTools
  ✓ Xorg_libXdmcp_jll
  ✓ fzf_jll
  ✓ Ogg_jll
  ✓ XZ_jll
  ✓ JpegTurbo_jll
  ✓ x265_jll
  ✓ x264_jll
  ✓ Zstd_jll
  ✓ libaom_jll
  ✓ LZO_jll
  ✓ Opus_jll
  ✓ Expat_jll
  ✓ Xorg_xtrans_jll
  ✓ Libiconv_jll
  ✓ Libffi_jll
  ✓ Wayland_protocols_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ Libgpg_error_jll
  ✓ Libuuid_jll
  ✓ FriBidi_jll
  ✓ Contour
  ✓ Missings
  ✓ Relocatable

In [ ]:
function pad(x::Vector, M::Int64)
    vcat(x, zeros(M - length(x)))
end

function seq_to_fun(x, M)
    fft(pad(x, M))
end

# Discrete prolate spheroidal sequences

The discrete prolate spheroidal sequences (Slepian, 1978), $v_n^{(k)}(N,W)$, are the eigenvector solutions to the eigenvalue problem

$$\sum_{m = 0}^{N-1}\frac{\sin 2\pi W(n-m)}{\pi(n-m)}v_m^{(k)}(N,W) = \lambda_k(N,W)\cdot v_n^{(k)}(N,W)$$

where $N$ is the length of the sequence, $n = 0, \ldots, N-1$, $W$ is the bandwidth, $K$ is the number of tapers, and $\lambda_k(N,W)$ is the eigenvalue corresponding to the $k$th taper. 

The dpss's solve the problem of concentrating, in frequency, the most amount of mass under the interval $(-W,W)$, while having only finite extent in time. 

In Slepians.jl, the dpss's are computed in the following way.

In [ ]:
N     = 1024
NW    = 4.0 
K     = 8
v,lam = dpss_tapers(N, NW, K, :both)

p1 = plot(v, label = round.(lam', digits=8), xlabel = "Index", 
    ylabel = "DPSS's sorted by concentration",
    xtick=([1,256,512,768,1024].-1))

The very first dpss taper has the largest "concentration" in the bandwidth of interest, very close to one. Subsequent tapers are less and less concentrated. To see this, plot their Fourier transforms as below. We zero pad, as before, to interpolate structure in frequency.

In [ ]:
M        = 4*N
halffreq = Int64(M/2-1)
U        = mapreduce(x -> abs2.(seq_to_fun(x, M)[2:halffreq,:]), hcat, v[:,k] for k in 1:K)
freq     = range(0, 1,length = M+1)[2:halffreq]

p2 = plot(freq, U, 
    label = round.(lam', digits=6), yscale = :log10, 
    xscale = :log10, xlims = [freq[1], freq[end]],
    xlabel = "Frequency (Hz)", 
    #title = "Slepian sequences",
    ylabel = "Squared magnitude response \n sorted by concentration",
    legend = :bottomleft, ylims =[1e-20,500],
    xtick=([1e-3,2e-3,5e-3,1e-2,2e-2,5e-2,1e-1,2e-1,5e-1],["0.001","0.002","0.005","0.01","0.02","0.05","0.1","0.2","0.5"])
)

plot!(p2, (NW/N)*ones(2), [1e-30, 1e3], label="W", line = :dash)

In [ ]:
plot!(p1,legend=false)
plot(p1, p2, size = [800,300])

One can see that the bulk of the mass under each of the curves is on the interval (-W,W). 
It is also easy to see that as the concentration, lambda, decreases, there is less mass under
the curve inside the band. Since the sequences all have unit energy, there is more mass outside
the band for larger index. Comparing with the tapers in the first plot, we see the power drop off rather quickly outside the band (-W, W) of interest. 

The bandwidth, $W$, is a tuning parameter for the multitaper method, and one adjusts the bandwidth by selecting the time-bandwidth product <tt>NW</tt> above. The number of tapers, $K$, is selected to be less than $2NW$, as the concentration rapidly drops with increasing $k$. For further reading, consult the original paper cited below. 

# References

Slepian, David. "Prolate spheroidal wave functions, Fourier analysis, and uncertainty—V: The discrete case." Bell System Technical Journal 57.5 (1978): 1371-1430.

In [ ]:
plot!(p1, legend=false, size = [400,300])
savefig(p1, "../paper/figures/Dpss_time.pdf")

In [ ]:
plot!(p2, size = [400,300], ylims = [1e-15,500],
xtick=([1e-3,2e-3,5e-3,1e-2,2e-2,5e-2,1e-1,2e-1,5e-1],["0.001","","","0.01","","","0.1","","0.5"])
)
savefig(p2,"../paper/figures/Dpss_freq.pdf")

In [ ]:
println(lam)

## Example - Durrani-Chapman

If one constrains to the following all-pole form for the magnitude response of a digital filter
$$ |H(f)|^2 = \frac{1}{1+\left| \sum_{j=1}^{N} \beta_k e^{i 2 pi f j} \right |^2} $$
Then as Durrani and Chapman showed, the coefficients $\beta_j$ is the $N$th (i.e. smallest) scaled discrete prolate spheroidal sequence, where the scaling factor is chosen so that the filter has its half power point at the chosen cutoff frequency $f_0$. Below, we recreate figures 1-3 from the example in the paper cited.

To begin, we alter the DPSS code so that it returns the Slepian sequence associated with the smallest eigenvalue instead of the largest ones.

In [ ]:
function dpss_matrix(nw, n)
    SymTridiagonal([cos(2*pi*(nw/n))*abs2(0.5*(n-1)-(j-1)) for j in 1:n],
                          [0.5*j*(n-j) for j in 1:(n-1)])
end

"""
    dpss_smallest(n, nw)

Computes the dpss associated with the smallest eigenvalue, normalized so that the first entry is 1.0.
"""
function dpss_smallest(n, nw) 
  stdm = dpss_matrix(nw, n)
  theta = eigvals(stdm, 1:1)
  vv = eigvecs(stdm, theta)
  return vv./vv[1]
end

In order to recreate Figs. 1 and 3, we use the following parameters, and we get the following matrix $\sigma$

In [ ]:
N = 4
NW = 0.1*N

sigma = dpss_matrix(NW, N)

We then obtain the dpss associated with the smallest eigenvalue, and normalize the vector so that the first entry is 1.0.

In [ ]:
vmin = dpss_smallest(N, NW)
println("Scaled eigenvector $(vmin)")
println("Eigenvalue $(eigvals(sigma, 1:1))")
plot(vmin, label = N)

We want to choose the constant $K$ so that $|H(f_0)|^2$ is 3 db down from $|H(0)|^2$
or
$$ K = \frac{1}{(|\phi(f_0)|^2 - 2|\phi(0)|^2)}$$
meanwhile, $K=\beta^T\beta$.

In [ ]:
# Find the scaling factor
K = (1.0 ./ (abs2.(exp.(-2*pi*1im*0.1*collect(1:N)')*vmin) - 2*abs2.(exp.(-2*pi*1im*0.0*collect(1:N)')*vmin)))[1,1]
beta = vmin * sqrt(K)

# Compute the transfer function
freq = LinRange(-0.5, 0.5, M+1)[1:M]
Hf = 1.0 ./ (1 .+ abs2.(exp.(-2*pi*1im*freq*collect(1:length(vmin))')*beta))

beta

In [ ]:
# Fig 1
p1 = plot(freq, 10*log10.(Hf), xlims=[0,0.5], label = N', ytick = -75:25:0, ylims = [-80,5], title = "Fig. 1")
plot!(p1, [1,1]*0.1, [-80,5], label = "W")


In [ ]:
function durranichapman_coeff(N, NW)
    vmin = dpss_smallest(N, NW)
    f0 = NW/N
    K = (1.0 / (abs2.(exp.(-2 * pi * 1.0im * f0 * collect(1:N)') * vmin)[1, 1] - 2 * abs2.(sum(vmin))))
    return vmin * sqrt(K)
end

function transfn(beta, M)
    # Compute the transfer function
    N = length(beta)
    freq = LinRange(-0.5, 0.5, M + 1)[1:M]
    Hf = 1.0 ./ (1 .+ abs2.(exp.(-2 * pi * 1im * freq * collect(1:N)') * beta))
    return Hf
end

In [ ]:
N = 4:2:10
NW = 0.1*N

beta = [durranichapman_coeff(N[i], NW[i]) for i in 1:4]
Hf = hcat([transfn(beta[i], M) for i in 1:4]...)

beta[1]

In [ ]:
# Fig 1
p1 = plot(freq, 10*log10.(Hf), xlims=[0,0.5], label = N', ytick = -75:25:0, ylims = [-80,5], title = "Fig. 1")
plot!(p1, [1,1]*0.1, 10*log10.([minimum(Hf), maximum(Hf)]), label = "W")
plot!(p1, [0,0.5],[1,1]*-3, label = "-3dB")

In [ ]:
plot!(p1, xlims=[0,0.12], ylims = [-10,5], ytick = -10:2.5:5 )

In [ ]:
# Fig 3
plot(freq, 10*log10.(Hf[:,1]), xlims=[0,0.12], ytick = -10:2.5:5, ylims = [-10,5], label = N[1], title = "Fig. 3")
plot!([1,1]*0.1, [-10,5], label = "W")
plot!([0,0.12],[1,1]*-3, label = "-3dB")

In [ ]:
f0 = [0.1, 0.2, 0.25, 0.3, 0.4]
N = 20
NW = f0*N

beta = [durranichapman_coeff(N, NW[i]) for i in 1:length(f0)]
Hf = hcat([transfn(beta[i], M) for i in 1:length(f0)]...)

# Fig 2
plot(freq, 10*log10.(Hf), xlims=[0,0.5], label = f0', ytick = -150:25:0, ylims = [-100,5], title = "Fig. 2")
plot!([1.,1] * f0', [-100,5], label = "")
plot!([0,0.5],[1,1]*-3, label = "-3dB")

## Reference
Durrani, T., and R. Chapman. "Optimal all-pole filter design based on discrete prolate spheroidal sequences." IEEE transactions on acoustics, speech, and signal processing 32.4 (1984): 716-721.